**Author**: Justine Debelius<br>
**email**: jdebelius@ucsd.edu<br>
**enviroment**: agp_2017<br>
**Date**: 27 April 2017

This notebook will take the current American Gut metadata from Qiita ([https://qiita.ucsd.edu/study/description/10317](https://qiita.ucsd.edu/study/description/10317)) and clean it up for analysis. The file from qiita should be downloaded into the same directory. Food Frequency Questionaire data from vioscreen was downloaded directly from the website, and compiled into a cleaned-up per-survey file in the previous notebook.

In [1]:
import re
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sn

% matplotlib inline

We'll start by loading the metadata file.

In [2]:
sample_ = pd.read_csv('./01.metadata/qiita_map/10317_20170426-140347.txt', sep='\t', dtype=str)

# Cleans up blanks

We're going to start by checking the blank values, and making sure we have consistent notation for the blanks. In this case, blanks will be denoted as `"not applicable"` as appropriate by identifying the blanks.

In [3]:
def check_blank(x):
    return "blank" in x.lower()

In [4]:
blanks = sample_['sample_name'].apply(check_blank)

Most columns we can simply specify the value as `"not applicable"`, but a handful have specific values for the blanks. We can specify these special columns first, and then fix anything that is not a special column.

In [5]:
special_missing = {'sample_name', 'description', 'altitude', 'depth',
                   'env_biome', 'env_feature', 'env_material', 'env_package', 
                   'longitude', 'latitude', 'elevation', 'country', 
                   'host_common_name', 'host_taxid', 'scientific_name',
                   'orig_name', 'geo_loc_name', 'state', 'alcohol_types', 
                   'allergic_to', 'mental_illness_type', 'non_food_allergies',
                   'depth', 'altitude', 'title', 'qiita_study_id',
                   'specialized_diet', 'public', 'pets_other_freetext'
                   }

regular_columns = [col for col in sample_.columns if col not in special_missing]
sample_.loc[blanks, regular_columns] = 'not applicable'

Then, we'll fix the special columns.

The depth and altitude should be 0 for all samples.

In [6]:
sample_.loc[blanks, 'altitude'] = 0
sample_.loc[blanks, 'depth']  = 0

We'll also make sure the ENVO ontology is listed consistently and up to date with the most recent Qiita standards.

In [7]:
sample_.loc[blanks, 'env_biome'] = 'urban biome'
sample_.loc[blanks, 'env_feature'] = 'reserach facility'
sample_.loc[blanks, 'env_material'] = 'sterile water'
sample_.loc[blanks, 'env_package'] = 'misc environment'

The `longitude`, `latitude`, `elevation`, `country`, `state`, and `geo_loc_name` are all listed correctly, so I'll leave those.

We can also fix the `scientific_name` and `host_taxid`.

In [8]:
sample_.loc[blanks, 'scientific_name'] = "freshwater metagenome"
sample_.loc[blanks, 'host_taxid'] = "449393"

# Standard Colums

There are a set of columns whcih should have consistent values across all samples. These include the altitude, depth, Qiita Study ID, and Title.

In [9]:
sample_['altitude'] = '0'
sample_['depth'] = '0'
sample_['qiita_study_id'] = '10317'
sample_['title'] = 'American Gut Project'
sample_['env_biome'] = 'urban biome'
sample_['public'] = 'true'

In [10]:
'10317.000051090' in sample_.index

False

# Boolean Columns

Next, we'll clean up the boolean columns, and make them consistent.

There are three major types of questions we need to account for. There's a problem with inconsistency in boolean questions, where True and False can also be represented as "Yes" and "No". We can write a simple function to fix this.

In [11]:
boolean_cols = ['acne_medication',
                'acne_medication_otc',
                'alcohol_consumption',
                'alcohol_types_beercider',
                'alcohol_types_red_wine',
                'alcohol_types_sour_beers',
                'alcohol_types_spiritshard_alcohol',
                'alcohol_types_unspecified',
                'alcohol_types_white_wine',
                'allergic_to_i_have_no_food_allergies_that_i_know_of',
                'allergic_to_other',
                'allergic_to_peanuts',
                'allergic_to_shellfish',
                'allergic_to_tree_nuts',
                'allergic_to_unspecified',
                'appendix_removed',
                'breastmilk_formula_ensure',
                'cat',
                'csection',
                'dna_extracted',
                'dog',
                'has_physical_specimen',
                'lactose',
                'lowgrain_diet_type',
                'mental_illness',
                'mental_illness_type_anorexia_nervosa',
                'mental_illness_type_bipolar_disorder',
                'mental_illness_type_bulimia_nervosa',
                'mental_illness_type_depression',
                'mental_illness_type_ptsd_posttraumatic_stress_disorder',
                'mental_illness_type_schizophrenia',
                'mental_illness_type_substance_abuse',
                'mental_illness_type_unspecified',
                'multivitamin',
                'nail_biter',
                'non_food_allergies_beestings',
                'non_food_allergies_drug_eg_penicillin',
                'non_food_allergies_pet_dander',
                'non_food_allergies_poison_ivyoak',
                'non_food_allergies_sun',
                'non_food_allergies_unspecified',
                'other_supplement_frequency',
                'pets_other',
                'physical_specimen_remaining',
                'pregnant',
                'public',
                'roommates_in_study',
                'seasonal_allergies',
                'softener',
                'specialized_diet_exclude_dairy',
                'specialized_diet_exclude_nightshades',
                'specialized_diet_exclude_refined_sugars',
                'specialized_diet_fodmap',
                'specialized_diet_halaal',
                'specialized_diet_i_do_not_eat_a_specialized_diet',
                'specialized_diet_kosher',
                'specialized_diet_modified_paleo_diet',
                'specialized_diet_other_restrictions_not_described_here',
                'specialized_diet_paleodiet_or_primal_diet',
                'specialized_diet_raw_food_diet',
                'specialized_diet_unspecified',
                'specialized_diet_westenprice_or_other_lowgrain_low_processed_fo',
                'subset_age',
                'subset_antibiotic_history',
                'subset_bmi',
                'subset_diabetes',
                'subset_healthy',
                'subset_ibd',
                'tonsils_removed']

In [12]:
def clean_up_bool(x):
    if x in {'No', 'false', 'False'}:
        return "No"
    elif x in {'Yes', 'true', 'True'}:
        return "Yes"
    else:
        return x

In [13]:
for col in boolean_cols:
    sample_[col] = sample_[col].apply(clean_up_bool)

# Multigroup questions

The next type of question which need to be addressed are the groups of questions. There are a series of connected boolean columns, represented by checkboxes, where a column in the group describes whether the question was left unanswered. We need to group these so the columns together so that the unanswered questions are marked.

In [14]:
grouped_columns = {
    'alcohol_types_unspecified' : ['alcohol_types_beercider',
                                   'alcohol_types_red_wine',
                                   'alcohol_types_sour_beers',
                                   'alcohol_types_spiritshard_alcohol',
                                   'alcohol_types_white_wine'],
    'allergic_to_unspecified': ['allergic_to_i_have_no_food_allergies_that_i_know_of',
                                'allergic_to_other',
                                'allergic_to_peanuts',
                                'allergic_to_shellfish',
                                'allergic_to_tree_nuts'],
    'mental_illness_type_unspecified': ['mental_illness_type_anorexia_nervosa',
                                        'mental_illness_type_bipolar_disorder',
                                        'mental_illness_type_bulimia_nervosa',
                                        'mental_illness_type_depression',
                                        'mental_illness_type_ptsd_posttraumatic_stress_disorder',
                                        'mental_illness_type_schizophrenia',
                                        'mental_illness_type_substance_abuse'],
    'non_food_allergies_unspecified': ['non_food_allergies_beestings',
                                       'non_food_allergies_drug_eg_penicillin',
                                        'non_food_allergies_pet_dander',
                                        'non_food_allergies_poison_ivyoak',
                                        'non_food_allergies_sun'],
    'specialized_diet_unspecified': ['specialized_diet_exclude_dairy',
                                     'specialized_diet_exclude_nightshades',
                                     'specialized_diet_exclude_refined_sugars',
                                     'specialized_diet_fodmap',
                                     'specialized_diet_halaal',
                                     'specialized_diet_i_do_not_eat_a_specialized_diet',
                                     'specialized_diet_kosher',
                                     'specialized_diet_modified_paleo_diet',
                                     'specialized_diet_other_restrictions_not_described_here',
                                     'specialized_diet_paleodiet_or_primal_diet',
                                     'specialized_diet_raw_food_diet',
                                     'specialized_diet_westenprice_or_other_lowgrain_low_processed_fo'
                                     ]
    }
group_drop = ['alcohol_types', 'allergic_to', 'mental_illness_type', 'non_food_allergies',
              'specialized_diet', 'pets_other_freetext']
vios_drop = [c for c in sample_.columns if 'vioscreen' in c]
group_drop.extend(vios_drop)

In [15]:
for unspecified_col, columns in grouped_columns.items():
    unsp = sample_[unspecified_col] == "Yes"
    sample_.loc[unsp, columns] = "Unspecified"

We'll also drop any column that represents a placeholder in the download.

In [16]:
sample_.drop(group_drop, axis='columns', inplace=True)

# Cleans up missing values

There are quite a few columns where missing values are simply specified as nans (likely because no value is provided there). We're going to correct this.

In [17]:
regular_columns = list(set(regular_columns) - set(group_drop))
for column in regular_columns:
    sample_.loc[pd.isnull(sample_[column]), column] = "Unspecified"

# Handles case specificity
sample_.replace('unspecified', 'Unspecified', inplace=True)

# Standardize required columns

The standards for the required columns have been updated during qiita. Essentially, our compliance with mimarks standards has changed as our understanding has evolved, this will standardize the values according to the best standard I can find now.

In [18]:
real = blanks == False

All of our samples are human samples, so they come from a host associated habitat in an urban biome, and their host common name should be "human".

In [19]:
sample_.loc[real, 'env_feature'] = 'human-associated habitat'
sample_.loc[real, 'host_common_name'] = 'human'

We'll also convert the scientific name so the description of hte location is consistient. We'll build this off the body habitat.

In [20]:
scientific_name_map = {'UBERON:ear': 'human metagenome',
                       'UBERON:eye': 'human eye metagenome',
                       'UBERON:feces': 'Homo sapiens sapiens',
                       'UBERON:feces': 'human gut metagenome',
                       'UBERON:hair': 'Homo sapiens sapiens',
                       'UBERON:hair': 'human skin metagenome',
                       'UBERON:nose': 'Homo sapiens sapiens',
                       'UBERON:nose': 'human nasal/pharyngeal metagenome',
                       'UBERON:oral cavity': 'Homo sapiens sapiens',
                       'UBERON:oral cavity': 'human oral metagenome',
                       'UBERON:skin': 'Homo sapiens sapiens',
                       'UBERON:skin': 'human skin metagenome',
                       'UBERON:vagina': 'Homo sapiens sapiens',
                       'UBERON:vagina': 'human vaginal metagenome',
                       'not applicable': 'freshwater metagenome',
                       }
sample_['scientific_name'] = sample_['body_habitat'].replace(scientific_name_map)

We'll infer the enviromental material based on the `body_product`.

In [21]:
def map_enviromental_material(x):
    return x.replace('UBERON:', '')

sample_.loc[sample_['env_material'] == 'Unspecified', 'env_material'] = \
    sample_.loc[sample_['env_material'] == 'Unspecified', 'body_product'].apply(map_enviromental_material)

We're also going to make the enviromental package consistent. This will require two steps. First, we'll drop an "Unspecified" values, because the env package cannot be unspecified. Then, we'll infer it from the body habitat.

In [22]:
env_package_lookup = {'UBERON:feces': 'human-gut',
                      'UBERON:skin': 'human-skin',
                      'UBERON:nose': 'human-skin',
                      'UBERON:oral cavity': 'human-oral',
                      'UBERON:vagina': 'human-vaginal',
                      'UBERON:hair': 'host-associated',
                      'UBERON:ear': 'host-associated',
                      'UBERON:eye': 'host-associated',
                      }
# Fixes the nan issue
sample_.loc[real, 'env_package'].replace('Unspecified', np.nan, inplace=True)
sample_.loc[pd.isnull(sample_['env_package']), 'env_package'] = \
    sample_.loc[pd.isnull(sample_['env_package']), 'body_habitat'].apply(lambda x: env_package_lookup[x])

# Categorical clean up

There are a few questions which need some clean up due to survey formatting issues. I'll tackle those here.

The first has to do with a mapping of stool frequency, where the introduction of the brisol scale has modified the question values returned.

In [23]:
stool_clean_up = {
    'I tend to have normal formed stool': 'I tend to have normal formed stool',
    'I tend to have normal formed stool - Type 3 and 4': 'I tend to have normal formed stool',
    'I tend to have diarrhea (watery stool) - Type 5, 6 and 7': 'I tend to have diarrhea (watery stool)',
    'I tend to have diarrhea (watery stool)': 'I tend to have diarrhea (watery stool)',
    'I tend to be constipated (have difficulty passing stool) - Type 1 and 2': 'I tend to be constipated (have difficulty passing stool)',
    'I tend to be constipated (have difficulty passing stool)': 'I tend to be constipated (have difficulty passing stool)',
    "I don't know, I do not have a point of reference": "I don't know, I do not have a point of reference",
    "Unspecified": "Unspecified",
    "not applicable": "not applicable"
    }
sample_['bowel_movement_quality'] = sample_['bowel_movement_quality'].apply(lambda x: stool_clean_up[x])

# Age and continous clean up

We also found an issue with age, so we're going to re-calculate the base age, based on birth year and age. We'll assume a birthdate of January 1 for everyone, given we have no birth date or month. 

In [24]:
def calculate_age(x):
    """Converts birth year and collection date to an approximate age"""
    col_date = x['collection_date']
    b_year = x['birth_year']
    
    if ((col_date in {'not applicable', 'Unspecified'}) or 
        (b_year in {'not applicable', 'Unspecified'})):
        return b_year
    
    c_year = pd.to_datetime(col_date).year
    b_year = int(float(b_year))
    diff = int(c_year - b_year)

    if diff < 0:
        return 'Unspecified'
    elif diff > 110:
        return "Unspecified"
    return int(diff)

We'll add the adjusted age.

In [25]:
sample_['age_corrected'] = sample_.apply(calculate_age, axis=1)

Given that we're already correcting the age, let's also correct the height and weight in this process, and re-calculate the height, weight, age_corrected, age_cat, bmi, and bmi_cat.

We've already placed a filter on the age so there can not be negative ages.

We can also check the height and weight. We'll gate height in that an average full term infant is about 51 cm, with 1 standard devation below at 48 cm [[My health Alberta](https://myhealth.alberta.ca/Health/Pages/conditions.aspx?hwid=te6295)]. We'll assume an upper height limit of 7 feet (210 cm). Since we are dealing with infants of 6 weeks or older and infants dont get shorter, we can set these as initial gates.

So, we'll initially drop these values.

In [26]:
def check_height(x):
    if x in {'not applicable', 'Unspecified'}:
        return x
    elif (float(x) < 48) | (210 < float(x)):
        return "Unspecified"
    else:
        return '%i' % float(x)

In [27]:
sample_['height_corrected'] = sample_['height_cm'].apply(check_height)

We'll set a second gate for weight that a full term infant is at least 2.5 kg, and no more than 200 kg (450 lbs).

In [28]:
   def check_weight(x):
    if x in {'not applicable', 'Unspecified'}:
        return x
    elif (float(x) < 2.5) | (200 < float(x)):
        return "Unspecified"
    else:
        return '%i' % float(x)

In [29]:
sample_['weight_corrected'] = sample_['weight_kg'].apply(check_height)

We'll also correct age so that anyone younger than 4 years old has a height of 105 cm, or a weight of 20kg, based on the [CDC growth chart](https://www.cdc.gov/growthcharts/data/set1clinical/cj41l021.pdf). We'll assume that if height or weight is not provided, they still are an infant. We'll also include a gate that young children should not drink alcohol.

In [55]:
def check_height(x):
    return not ((x in {'not applicable', 'Unspecified'}) or
                (float(x) < 105))
        
def check_weight(x):
    return not ((x in {'not applicable', 'Unspecified'}) or 
                (float(x) < 20))

def check_etoh(x):
    return not ((x in {'not applicable', 'Unspecified'}) or
                (x == 'No'))

In [56]:
def correct_age(x):
    age = x['age_corrected']
    if age in {'not applicable', 'Unspecified'}:
        return age

    age = float(age)
    if age > 3:
        return '%i' % age

    height = check_height(x['height_corrected'])
    weight = check_weight(x['weight_corrected'])
    etoh = check_etoh(x['alcohol_consumption'])
    
    if (height | weight | etoh):
        return 'Unspecified'
    else:
        return '%i' % age

In [57]:
sample_['age_corrected'] = sample_.apply(correct_age, axis=1)

We'll also gate height and weight under the assumption that adults should be at least 105 cm and at least 20 kg.

In [58]:
def correct_size(x, size_cat, size_threshhold, age_threshhold=3):
    height = x[size_cat]
    age = x['age_corrected']
    if height in {'not applicable', 'Unspecified'}:
        return height
    elif ((float(height) < size_threshhold) and 
          (age not in {'not applicable', 'Unspecified'}) and
          (float(age) > age_threshhold)):
        return "Unspecified"
    else:
        return '%i' % float(height)

In [59]:
sample_['height_corrected'] = sample_.apply(correct_size, axis=1, args=['height_corrected', 105, 3])
sample_['weight_corrected'] = sample_.apply(correct_size, axis=1, args=['weight_corrected', 20, 3])

We can calculate the BMI for adults, and then cateogrize the BMI.

In [60]:
def calculate_bmi(x):
    height = x['height_corrected']
    weight = x['weight_corrected']
    
    if ((height == 'not applicable') | (weight == 'not applicable')):
        return 'not applicable'
    elif ((height == 'Unspecified') | (weight == 'Unspecified')):
        return "Unspecified"
    else:
        return float(weight) / np.square(float(height) / 100)

In [61]:
sample_['bmi_corrected'] = sample_.apply(calculate_bmi, axis=1)

Finally, we need to categorize the BMI and age, based on the corrected value.

In [62]:
def categorize_bmi(x):
    age = x['age_corrected']
    bmi = x['bmi_corrected']
    if ((age == 'not applicable') | (bmi == 'not applicable')):
        return 'not applicable'
    elif ((age == 'Unspecified') | (bmi == 'Unspecified')):
        return 'Unspecified'
    age = float(age)
    bmi = float(bmi)
    
    if (age < 18):
          return "Unspecified"
        
    if bmi < 18.5:
          return "Underweight"
    elif bmi < 25:
          return "Normal"
    elif bmi < 30:
          return "Overweight"
    else:
         return "Obese"

In [63]:
sample_['bmi_cat'] = sample_.apply(categorize_bmi, axis=1)

In [64]:
def categorize_age(x):
    if x in {'not applicable', 'Unspecified'}:
        return x
    x = float(x)
    if x < 3:
        return "Baby"
    elif x < 13:
        return "Child"
    elif x < 20:
        return "Teen"
    elif x < 30:
        return "20s"
    elif x < 40:
        return "30s"
    elif x < 50:
        return "40s"
    elif x < 60:
        return "50s"
    elif x < 70:
        return "60s"
    else:
        return "70+"

In [65]:
sample_['age_cat'] = sample_['age_corrected'].apply(categorize_age)

# Saves the file
We can save the updated, cleaned mapping file to a text file, to track the progress.

In [66]:
sample_.set_index('sample_name', inplace=True)
sample_.to_csv('./01.metadata/ag_sample_jwd.txt', sep='\t', index_label='sample_name')

# Adds vioscreen data

We'll next add the vioscreen data to the sample template. We'll start by loading the vioscreen data.

In [68]:
reports = pd.read_csv('./01.metadata/vioscreen_nutrient_report.txt', sep='\t', dtype=str)
reports.set_index('survey_id', inplace=True)

Next, we'll build a table that relates the barcode to the survey id.

In [69]:
map_ = sample_.reset_index()[['sample_name', 'survey_id']]
map_.set_index('survey_id', inplace=True)

In [70]:
map_.shape

(17004, 1)

In [71]:
vioscreen = map_.join(reports)
vioscreen.reset_index(inplace=True)
vioscreen.set_index('sample_name', inplace=True)

Finally, we'll update the blanks, so they are replace appropriately, and then save the vioscreen only data.

In [72]:
vios_blanks = vioscreen.loc[vioscreen['survey_id'] == 'not applicable'].index
vioscreen.loc[vios_blanks] = 'not applicable'

In [73]:
vioscreen.dropna().to_csv('./01.metadata/vioscreen_with_barcode.txt', sep='\t', index_label='sample_name')

Then, we'll replace the missing values, and save the vioscreen data alone.

In [74]:
vioscreen.replace(np.nan, 'Unspecified', inplace=True)
vioscreen.to_csv('./01.metadata/vioscreen_with_barcode_all_samples.txt', sep='\t', index_label='sample_name')

Finally, we'll drop the survey id, and join the data.

In [75]:
vioscreen.drop('survey_id', axis='columns', inplace=True)

And, finally, we'll combine the files.

In [76]:
sample_w_vios = pd.concat([sample_, vioscreen], axis=1)

In [77]:
sample_w_vios.replace(np.nan, 'not applicable', inplace=True)

sample_w_vios.to_csv('./01.metadata/ag_metadata_with_vioscreen.txt', sep='\t', index_label="sample_name")

# Adds Prep data

Next, we'll load the prep files for the American Gut data. These represent the multiple sequencing runs.

In [79]:
prep_dir = './01.metadata/qiita_preps/'
prep_fps = [os.path.join(prep_dir, fp_) for fp_ in os.listdir(prep_dir)]
prep_ = pd.concat([pd.read_csv(fp_, sep='\t', dtype=str) for fp_ in prep_fps])

In [80]:
prep_.reset_index(inplace=True)

We'll start by dropping the blank column from the preps.

In [81]:
prep_.drop(['Unnamed: 0', 'index'], axis='columns', inplace=True)

We'll replace any blanks in the file with "Unspecified".

In [82]:
prep_.replace(np.nan, "Unspecified", inplace=True)

In [83]:
prep_.shape

(18669, 43)

We also need to adjust the sample id to account for multiple runs of the same sample. To do this, we'll first determine how many prep templates a sample appears in. If a sample appears in one prep template, nothing will be done to modify the sample name.

If a prep appears in multiple templates, then the sample name will be suffixed with A, B, C, etc following the run date order.

In [91]:
num_sample_prep = prep_['sample_name'].value_counts()

In [92]:
one_replicate = num_sample_prep.index[num_sample_prep == 1]
multi_replicates = num_sample_prep.index[num_sample_prep > 1]

In [105]:
with open('/Users/jdebelius/Downloads/lost-samples.txt', 'r') as f_:
    yoshiki = set([id_ for id_ in f_.read().split('\n')]) - set([''])

In [107]:
yoshiki.issubset(set(multi_replicates))

True

We'll create an `adjusted_sample_name` column, which will allow us to index using the replicates.

In [93]:
for sample_name in one_replicate:
    prep_.loc[prep_['sample_name'] == sample_name, 'adj_sample_name'] = sample_name

In [94]:
for sample_name in multi_replicates:
    run_date = pd.to_datetime(prep_.loc[prep_['sample_name'] == sample_name, 'run_date'])
    run_date.sort_values(inplace=True)
    for (i, let_) in zip(*[run_date.index, 'ABCDEFG']):
        prep_.loc[i, 'adj_sample_name'] = '%s%s' % (sample_name, let_)

Let's save the merged prep template.

In [95]:
prep_.set_index('sample_name', inplace=True)
prep_.to_csv('./01.metadata/ag_prep_jwd.txt', sep='\t', index_label='sample_name')

Now, we can merge the sample and prep data. We will need to address the duplicate run of one sample.

In [96]:
prep_.loc['10317.000002418B'] = prep_.loc['10317.000002418'].copy()
prep_.rename(index={'10317.000002418': '10317.000002418A'}, inplace=True)

We'll join the two dataframes, and set the adjusted sample name as the index.

In [114]:
map_ = prep_.join(sample_w_vios)
map_.reset_index(inplace=True)

In [117]:
map_.rename(columns={'index': 'original_sample_name'}, inplace=True)
map_.set_index('adj_sample_name', inplace=True)

Finally, we'll save the combined map.

In [121]:
map_.to_csv('./01.metadata/ag_full_map.txt', sep='\t', index_label='#SampleID')

We've now generated a sample map that can be used in further analysis.